In [1]:
import google.auth
from googleapiclient.discovery import build
import datetime

In [ ]:
P = ! gcloud config list --format 'value(core.project)'
PROJECT_ID = P[0]
PROJECT_ID

In [10]:
def service_account_key_expiration_monitor(PROJECT_ID):
    credentials, project = google.auth.default()
    service = build('iam', 'v1', credentials=credentials)

    now = datetime.datetime.utcnow().date()
    alert_threshold = (now + datetime.timedelta(days=90))

    # List service accounts in the project
    accounts = service.projects().serviceAccounts().list(name=f'projects/{PROJECT_ID}').execute()

    expiring_keys = []

    for account in accounts.get('accounts', []):
        # List keys for each service account
        keys = service.projects().serviceAccounts().keys().list(name=account['name']).execute()

        for key in keys.get('keys', []):
            valid_before_time = datetime.datetime.strptime(key['validBeforeTime'], '%Y-%m-%dT%H:%M:%SZ').date()

            if valid_before_time <= alert_threshold:
                message = f"Service account key {key['name']} will expire on {valid_before_time}."
                expiring_keys.append(message)

    if expiring_keys:
        return '\n'.join(expiring_keys)
    else:
        return "No keys are set to expire within the next 90 days."

In [11]:
service_account_key_expiration_monitor(PROJECT_ID)

'No keys are set to expire within the next 90 days.'